In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
base = '/kaggle/input/cleaning/cleaned_out/'
names = ['matches','teamwise_home_and_away','deliveries','most_runs_average_strikerate','teams','players']
for i,name in enumerate(names):
    my_code = name+'='+'pd.read_pickle("'+base+name+'.pkl'+'")'
    exec(my_code)

In [ ]:
matches_full = pd.read_pickle("../input/ipl-eda/matches_world_eda.pkl")

## Let's first process some of the featuress we have

In [ ]:
cols_to_drop = ["toss_winner","toss_decision",]
for col in cols_to_drop:
    if col in matches_full.columns:
        matches_full.drop(columns=col,inplace=True)

In [ ]:
matches_full.info()

### Next let's apply encoding for the features

> Frequency encoding is useful for features with high cardinality while label encoding is for other columns (in most cases)

In [ ]:
cat_columns =  list(matches_full.select_dtypes(include=['object']).columns)
for col in cat_columns:
    print("col {} has {} unique values".format(col,matches_full[col].nunique()))

*We can encode the umpires by frequency and others by label_encoder*

In [ ]:
matches_engineering = matches_full.copy()

In [ ]:
def freq_encode_col(df,col):
    enc_nom_1 = (df.groupby(col).size()) / len(df)
    return df[col].apply(lambda x : enc_nom_1[x])
def freq_encode_cols(df,cols):
    for col in cols:
        df[col] = freq_encode_col(df,col)
    return df

In [ ]:
freq_enc_cols = ['umpire1','umpire2','umpire3']
label_enc_cols = [col for col in cat_columns if col not in freq_enc_cols]

In [ ]:
labelencoder = LabelEncoder()
for col in label_enc_cols:
    matches_engineering[col] = labelencoder.fit_transform(matches_engineering[col])

In [ ]:
matches_engineering = freq_encode_cols(matches_engineering,freq_enc_cols)

In [ ]:
matches_engineering.head()

In [ ]:
matches_engineering.info()

# Let's list our new features (inspired from EDA):
    * number of previous matches per pair teams
    * percentage of exponential weighted average of
        * previous matches played in a country (team1 vs team2)
        * previous matches won in a country (team1 vs team2)
        * previous matches played in a city (team1 vs team2)
        * previous matches won in a city (team1 vs team2)
        * previous winned matches 
        * previous winned matches per season
        * previous win rate matches per season
        * win rate of team1 vs team 2

In [ ]:
matches_play = matches_engineering.copy()


In [ ]:
matches_play_t1 = matches_play.drop(columns='team2').rename(columns={'team1':'team'})
win_index = list(matches_play_t1.columns).index('winner')
team_index = list(matches_play_t1.columns).index('team')
matches_play_t1['winner'] = matches_play_t1.apply(lambda x:int(x[win_index]==x[team_index]),axis=1)

In [ ]:
matches_play_t2 = matches_play.drop(columns='team1').rename(columns={'team2':'team'})
win_index = list(matches_play_t1.columns).index('winner')
team_index = list(matches_play_t1.columns).index('team')
matches_play_t2['winner'] = matches_play_t2.apply(lambda x:int(x[win_index]==x[team_index]),axis=1)

In [ ]:
matches_play_t1_t2 = matches_play_t1.append(matches_play_t2, ignore_index=True)
matches_play_t1_t2.sort_values(by='date',axis=0,inplace=True)

### Let's see the moving average and the rolling (for means, sums and stds) to see which gives more variablility

In [ ]:
df = matches_play_t1_t2.copy()
df = df.set_index(['date'])
years_num = 2
years_days = str(years_num*12*30)
k1 = df.groupby('team').rolling(years_days+'D')['winner'].mean()
k2 = df.groupby('team')['winner'].ewm(alpha=0.05).mean()
k1[5].plot()
k2[5].plot()

In [ ]:
k1

In [ ]:
k2[10].plot()
k2[0].plot()

> We can see that the data is noisy at first (as we are dealing with binary data), but the ewm shows the details more

In [ ]:
k1 = df.groupby('team').rolling(years_days+'D')['winner'].sum()
k1[0].plot()
k1[9].plot()

In [ ]:
k1[0].plot()
k1[10].plot()

> The sum shows the same trend for the different teams (with different levels), and help distingush those who appears on random time periods

> but we have the problem of having values in the first period not very reliable, so we will work with ratios between the two teams (we may try to truncate season 2008 too)

In [ ]:
k1 = df.groupby('team').rolling(years_days+'D')['winner'].std()
k1[10].plot()
k1[0].plot()

> The std for the teams is parely 0.5 for all teams (binary data), so we may calculate the std of the ewm (or sum) of the calculated features

In [ ]:
k1 = df.groupby('team').rolling(years_days+'D')['winner'].sum()

In [ ]:
k1 = df.groupby('team').rolling(years_days+'D')['winner'].sum()
k1 = k1.reset_index()
k1 = k1.set_index('date')
k2 = k1.groupby('team')['winner'].rolling(years_days+'D').std()
k2[0].plot()
k2[7].plot()

> Now this may be a good feature

> Places with discontinuities are for some teams that stopped playing for two seasons then returned

### let's check the duals win rate

In [ ]:
df = matches_play_t1.copy()
df = df.set_index('date')
years_num = 2
years_days = str(years_num*12*30)
k1 = df.groupby('duals_sets')['winner'].ewm(alpha=0.05).mean()

In [ ]:
df = matches_play_t2.copy()
df = df.set_index('date')
k2 = df.groupby('duals_sets')['winner'].ewm(alpha=0.05).mean()

In [ ]:
k1[0].plot()
k2[0].plot()

In [ ]:
k1[1].plot()
k2[1].plot()

> Some teams becomes better against other teams with time (this may gives good info for the model)

In [ ]:
k1[1].rolling(years_num).std().plot()

> The std will give info about how will the statistic we are using is reliable

In [ ]:
k1 = k1.reset_index()
k1

In [ ]:
df1 = df.reset_index()

In [ ]:
df2 = df1.merge(k1, on=['duals_sets','date'])
groups = df2.groupby("winner_x")
for name, group in groups:
    plt.plot(group["date"], group["winner_y"], marker="o", linestyle="", label=name)
plt.legend()

In [ ]:
df = matches_play_t1.copy()
df = df.set_index('date')
years_num = 2
years_days = str(years_num*12*30)
k1 = df.groupby(['duals_sets','city'])['winner'].ewm(alpha=0.05).mean()

In [ ]:
k1 = k1.reset_index()
k1.head()

In [ ]:
df2 = df1.merge(k1, on=['duals_sets','date'])
groups = df2.groupby("winner_x")
for name, group in groups:
    plt.plot(group["date"], group["winner_y"], marker="o", linestyle="", label=name)
plt.legend()

> We can see a great seperator here when grouping by country too
> It looks like many small teams playing in some countries for the first time are not match for other teams who play in these countries always

In [ ]:
df = matches_play_t1.copy()
df = df.set_index('date')
years_num = 2
years_days = str(years_num*12*30)
k1 = df.groupby(['duals_sets','country'])['winner'].ewm(alpha=0.05).mean()

k1 = k1.reset_index()
df2 = df1.merge(k1, on=['duals_sets','date'])
groups = df2.groupby("winner_x")
for name, group in groups:
    plt.plot(group["date"], group["winner_y"], marker="o", linestyle="", label=name)
plt.legend()

In [ ]:
df = matches_play_t1.copy()
df = df.set_index(['date','id'])
years_num = 2
years_days = str(years_num*12*30)
k1 = df.groupby(['duals_sets','country'])['winner'].ewm(alpha=0.05).mean()

k1 = k1.reset_index()
df2 = df1.merge(k1, on=['duals_sets','date'])
groups = df2.groupby("winner_x")
for name, group in groups:
    plt.plot(group["date"], group["winner_y"], marker="o", linestyle="", label=name)
plt.legend()

In [ ]:
df = matches_play_t1.copy()


In [ ]:
matches_play_t1.date.nunique()

In [ ]:
def get_same_day_matches_id(s):
    k = np.arange(len(s))
    return k
def add_same_day_mathches_id(df,group_cols):
    cols = list(df.columns)
    for c in group_cols:
        cols.remove(c)
    col = cols[0]
    return df.groupby(group_cols)[col].transform(get_same_day_matches_id).values

In [ ]:
matches_play_t1['same_day_mathches_id'] = add_same_day_mathches_id(matches_play_t1,['duals_sets','date'])
sns.countplot(matches_play_t1['same_day_mathches_id'])

In [ ]:
def create_rolling_feat(df,groups_input,roll_col,col_name,key_cols,is_ewm=True,func_to_apply='mean',years_num=2,alpha=0.05):
    # preprcess the dataframe
    df_input = df.copy()
    df_input = df_input.set_index('date')
    groups = deepcopy(groups_input)
    years_days = str(years_num*12*30)
    # get the group
    if is_ewm:
        k1 = df_input.groupby(groups)[roll_col].ewm(alpha=alpha)
    else:
        k1 = df_input.groupby(groups).rolling(years_days+'D')[roll_col]
    # apply the function
    if func_to_apply == 'mean':
        k1 = k1.mean()
    elif func_to_apply == 'sum':
        k1 = k1.sum()
    elif func_to_apply == 'std':
        k1 = k1.std()
    
    k1.name = col_name
    # prepare df to merge
    df_input = df_input.reset_index()
    groups.extend(['date','same_day_mathches_id'])
    # prepare k1 to merge
    k1 = k1.reset_index()
    k1 ['same_day_mathches_id'] = add_same_day_mathches_id(k1,key_cols)
    # merge
#     print(df_input.columns)
#     print(k1.columns)
    df_input = df_input.merge(k1,how='left', on=groups)
#     print(df_input.shape)
    
    return df_input
    

In [ ]:
from copy import deepcopy
roll_col = 'winner'
groups_cols = [['country'],['venue'],['city'],['Season']]
groups_cols1 =deepcopy(groups_cols)
[e.insert(0,'duals_sets')for e in groups_cols1]
# groups_cols1.append(['duals_sets'])
groups_cols1

In [ ]:
k = len(groups_cols1)
groups_cols2 = deepcopy(groups_cols1)
for i in range(k):
    ele = groups_cols2[i]
    ele[0] = 'team'
    groups_cols1.append(ele)
groups_cols1    

In [ ]:
groups_cols1.extend([['duals_sets'],['team']])

In [ ]:
df = matches_play_t1.copy()



for group in groups_cols1:
    roll_col = 'winner'
    # ewm mean columns
    if group[0] == 'team':
        key_cols = ['team','date']
    else:
        key_cols = ['duals_sets','date']
    new_col = '_'.join(group)+'_ewm_mean'
    df = create_rolling_feat(df,group,roll_col,new_col,key_cols)
    for i in [1,2,3,4]:
        new_col = '_'.join(group)+'_sum_'+str(i)
        df = create_rolling_feat(df,group,roll_col,new_col,key_cols,is_ewm=False,func_to_apply='sum',years_num=i)
        
        
    
    


In [ ]:
df = df.T.drop_duplicates().T
df.head()

In [ ]:
df

In [ ]:
df.to_csv('matches_fat_eng.csv')